## Convolution Layer

- 输入：Tensor (minibatch, in_channels, iH, iW)
- 输出：Tensor (minibatch, out_channels, oH, oW)
- 参数：
    - out_channels
    - in_channels
    - kernel_size
    - stride：kernel在输入上的滑动步长
    - padding：在输入的四周填充0的层数
    - padding_mode：'zeros', 'reflect', 'replicate' or 'circular'
    - dilation：kernel元素之间的间隔
    - groups：分组卷积
    - bias：是否使用偏置

根据前后两层的尺寸关系，可以计算出padding和stride的值。
官方文档中提供了计算的公式。


In [13]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [8]:
input = torch.tensor([
    [1, 2, 0, 3, 1],
    [0, 1, 2, 3, 1],
    [1, 2, 1, 0, 0],
    [5, 2, 3, 1, 1],
    [2, 1, 0, 1, 1]
])
kernel = torch.tensor([[1, 2, 1],
                       [0, 1, 0],
                       [2, 1, 0]])

# input = input.view(1, 1, 5, 5).float()
input = torch.reshape(input, (1, 1, 5, 5))
kernel = torch.reshape(kernel, (1, 1, 3, 3))

input.shape, kernel.shape

(torch.Size([1, 1, 5, 5]), torch.Size([1, 1, 3, 3]))

In [5]:
output = torch.nn.functional.conv2d(input, kernel, stride=1)
output.shape, output

(torch.Size([1, 1, 3, 3]),
 tensor([[[[10, 12, 12],
           [18, 16, 16],
           [13,  9,  3]]]]))

In [6]:
output2 = torch.nn.functional.conv2d(input, kernel, stride=2)
output2.shape, output2

(torch.Size([1, 1, 2, 2]),
 tensor([[[[10, 12],
           [13,  3]]]]))

## Max Pooling Layer

- 输入：Tensor (minibatch, in_channels, iH, iW)
- 输出：Tensor (minibatch, out_channels, oH, oW)
- 参数：
    - kernel_size
    - stride：kernel在输入上的滑动步长
    - padding：在输入的四周填充0的层数
    - dilation：kernel元素之间的间隔
    - return_indices：是否返回最大值的索引
    - ceil_mode：移动步长之后，如果元素不足一个kernel_size，保留，默认不保留

In [11]:
output3 = torch.nn.functional.max_pool2d(input, kernel_size=3, ceil_mode=True)
output3.shape, output3

(torch.Size([1, 1, 2, 2]),
 tensor([[[[2, 3],
           [5, 1]]]]))

In [18]:
class Yukiiiii(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=3, ceil_mode=False)
        
    def forward(self, x):
        return self.maxpool1(x)
    
dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

net = Yukiiiii()

writer = SummaryWriter("module")
for epoch in range(2):
    step = 0
    for data in dataloader:
        imgs, targets = data
        writer.add_images(f"before epoch: {epoch}", imgs, step)
        out = net(imgs)  
        writer.add_images(f"epoch: {epoch}", out, step)
        step += 1
writer.close()

Files already downloaded and verified
